In [1]:
import carla
import numpy as np
import cv2
import time
import sys
import math
sys.path.append('F:\CARLA\Windows\CARLA_0.9.15\PythonAPI\carla') # tweak to where you put carla
from agents.navigation.global_route_planner import GlobalRoutePlanner

In [2]:
client = carla.Client('localhost', 2000)

world = client.get_world()

blueprints = [bp for bp in world.get_blueprint_library().filter('*')]

spawn_points = world.get_map().get_spawn_points()
print(spawn_points)

[<carla.libcarla.Transform object at 0x000002B2A6943C00>, <carla.libcarla.Transform object at 0x000002B2A6943E30>, <carla.libcarla.Transform object at 0x000002B2A6943DC0>, <carla.libcarla.Transform object at 0x000002B2A6943F10>, <carla.libcarla.Transform object at 0x000002B2A6B7A030>, <carla.libcarla.Transform object at 0x000002B2A6B7A0A0>, <carla.libcarla.Transform object at 0x000002B2A6B7A110>, <carla.libcarla.Transform object at 0x000002B2A6B7A180>, <carla.libcarla.Transform object at 0x000002B2A6B7A1F0>, <carla.libcarla.Transform object at 0x000002B2A6B7A260>, <carla.libcarla.Transform object at 0x000002B2A6B7A2D0>, <carla.libcarla.Transform object at 0x000002B2A6B7A340>, <carla.libcarla.Transform object at 0x000002B2A6B7A3B0>, <carla.libcarla.Transform object at 0x000002B2A6B7A420>, <carla.libcarla.Transform object at 0x000002B2A6B7A490>, <carla.libcarla.Transform object at 0x000002B2A6B7A500>, <carla.libcarla.Transform object at 0x000002B2A6B7A570>, <carla.libcarla.Transform obje

In [3]:
def destroy():#Destroying the existing things
    for actor in world.get_actors().filter('*vehicle*'):
        actor.destroy()
    for sensor in world.get_actors().filter('*sensor*'):
        sensor.destroy()

In [4]:
def DrawPointsFor30Sec(world, spawn_points):
    drawn_points = []
    for index, waypoint in enumerate(spawn_points):
        # Draw a string with an ID at the location of each spawn point
        point_id = f'ID: {index}'
        point = world.debug.draw_string(
            waypoint.location,
            point_id,
            draw_shadow=False,
            color=carla.Color(r=255, g=255, b=255),
            life_time=30,  # Set to 0 to make it persist indefinitely
            persistent_lines=True
        )
        drawn_points.append(point)

In [5]:
DrawPointsFor30Sec(world, spawn_points)

Get the bicycle

In [6]:
def bicycleorigin():
    global bicycle
    bicycle_bp = world.get_blueprint_library().filter('*crossbike*')
    bicycle_start_point = spawn_points[1]

    bicycle = world.try_spawn_actor(bicycle_bp[0], bicycle_start_point)
    bicyclepos = carla.Transform(bicycle_start_point.location + carla.Location(x=-3, y=3.5))
    bicycle.set_transform(bicyclepos)

Get the car

In [7]:
def carorigin():
    global vehicle, vehicle_start_point
    vehicle_bp = world.get_blueprint_library().filter('*mini*')
    vehicle_start_point = spawn_points[94]
    vehicle = world.try_spawn_actor(vehicle_bp[0], vehicle_start_point)

Define the Car speed and etc specifications

In [8]:
vehicle = None
bicycle = None
vehicle_start_point = None
destroy()
bicycleorigin()
carorigin()

In [9]:
PREFERRED_SPEED = 15 #km/h
SPEED_THRESHOLD = 2 #When we close to the required speed, it drops the throttle
MAX_STEER_DEGREES = 40

#Params for displaying text
font = cv2.FONT_HERSHEY_SIMPLEX

org = (30, 30) # This is for current speed
org2 = (30, 50) #This is for future steering angle
org3 = (30, 70) #this is for future telemetry
org4 = (30, 90) #this is for future telemetry
org5 = (30, 110) #this is for future telemetry
fontscale = 0.5
color = (255, 255, 255)
tickness = 1

def maintain_speed(s: float) -> float:
    
    if s >= PREFERRED_SPEED:
        return 0
    elif s < PREFERRED_SPEED - SPEED_THRESHOLD:
        return 0.8
    else:
        return 0.4


Setup the collision detector

In [10]:
def process_collision(event):
    # Extract collision data
    global collision_happened
    other_actor = event.other_actor
    impulse = event.normal_impulse
    collision_location = event.transform.location
    print(f"Collision with {other_actor.type_id}")
    print(f"Impulse: {impulse}")
    print(f"Location: {collision_location}")
    collision_happened = True

In [11]:
collision_detector_bp = world.get_blueprint_library().find('sensor.other.collision')
collision_sensor = world.spawn_actor(
        collision_detector_bp,
        carla.Transform(),
        attach_to=vehicle
    )
collision_sensor.listen(lambda event: process_collision(event))


Setup the GlobalRoutePlanner

In [12]:
targetid = 27
targetPoint = spawn_points[targetid]

point_A = vehicle_start_point.location
point_B = targetPoint.location


sampling_resolution = 1
grp = GlobalRoutePlanner(world.get_map(), sampling_resolution)

route = grp.trace_route(point_A, point_B)
for waypoint in route:
    world.debug.draw_string(waypoint[0].transform.location, '^', draw_shadow=False,
                            color=carla.Color(r=0, g=0, b=255), life_time=600.0,
                            persistent_lines=True)

In [13]:
def angle_between(v1, v2):
    return math.degrees(np.arctan2(v1[1], v1[0]) - np.arctan2(v2[1], v2[0]))

def get_angle(car, wp):
    vehicle_pos = car.get_transform()
    car_x = vehicle_pos.location.x
    car_y = vehicle_pos.location.y
    wp_x = wp.transform.location.x
    wp_y = wp.transform.location.y


    #vector to waypoint
    x = (wp_x - car_x)/((wp_y - car_y)**2 + (wp_x - car_x)**2)**0.5
    y = (wp_y - car_y)/((wp_y - car_y)**2 + (wp_x - car_x)**2)**0.5


    #car vector
    car_vector = vehicle_pos.get_forward_vector()
    degrees = angle_between((x,y), (car_vector.x, car_vector.y))

    return degrees

Setup the Camera sensor and the YOLO algoritm

In [14]:
from ultralytics import YOLO
#setup the camera
CAMERA_POS_Z = 1.5 
CAMERA1_POS_X = 0
CAMERA2_POS_X = 1
CAMERA1_POS_Y = 0.5
CAMERA2_POS_Y = 1.5

camera_bp = world.get_blueprint_library().find('sensor.camera.rgb')
camera_bp.set_attribute('image_size_x', '640') # this ratio works in CARLA 9.14 on Windows
camera_bp.set_attribute('image_size_y', '360')


rightcamera1_init_trans = carla.Transform(carla.Location(z=CAMERA_POS_Z,x=CAMERA1_POS_X, y = CAMERA1_POS_Y), carla.Rotation(yaw=90))
rightcamera1 = world.spawn_actor(camera_bp,rightcamera1_init_trans,attach_to=vehicle)

rightcamera2_init_trans = carla.Transform(carla.Location(z=CAMERA_POS_Z,x=CAMERA2_POS_X, y = CAMERA1_POS_Y), carla.Rotation(yaw=90))
rightcamera2 = world.spawn_actor(camera_bp,rightcamera2_init_trans,attach_to=vehicle)

frontcamera1_init_trans = carla.Transform(carla.Location(z=CAMERA_POS_Z,x=CAMERA1_POS_X, y = CAMERA1_POS_Y))
frontcamera1 = world.spawn_actor(camera_bp,frontcamera1_init_trans,attach_to=vehicle)

frontcamera2_init_trans = carla.Transform(carla.Location(z=CAMERA_POS_Z,x=CAMERA1_POS_X, y = CAMERA2_POS_Y))
frontcamera2 = world.spawn_actor(camera_bp,frontcamera2_init_trans,attach_to=vehicle)

def camera_callback(image,data_dict):
    data_dict['image'] = np.reshape(np.copy(image.raw_data),(image.height,image.width,4))

image_w = camera_bp.get_attribute('image_size_x').as_int()
image_h = camera_bp.get_attribute('image_size_y').as_int()

rightcamera1_data = {'image': np.zeros((image_h,image_w,4))}
rightcamera2_data = {'image': np.zeros((image_h,image_w,4))}
frontcamera1_data = {'image': np.zeros((image_h,image_w,4))}
frontcamera2_data = {'image': np.zeros((image_h,image_w,4))}
# this actually opens a live stream from the camera
rightcamera1.listen(lambda image: camera_callback(image,rightcamera1_data))
rightcamera2.listen(lambda image: camera_callback(image,rightcamera2_data))
frontcamera1.listen(lambda image: camera_callback(image,frontcamera1_data))
frontcamera2.listen(lambda image: camera_callback(image,frontcamera2_data))
model = YOLO("best.pt")

c:\Users\mihge\anaconda3\envs\carla-sim\lib\site-packages\requests\__init__.py:114: RequestsDependencyWarning: urllib3 (1.26.14) or chardet (None)/charset_normalizer (3.1.0) doesn't match a supported version!
  RequestsDependencyWarning,


Match the objects

In [ ]:
import math

def match_bicycles_between_left_right(bicycles_left: list, bicycles_right: list):
    image_w = 640  # Image width
    fov = 90  # Field of view in degrees
    baseline = 1  # Baseline distance in meters
    focal_length = image_w / (2 * math.tan(math.radians(fov / 2)))  # Focal length in pixels

    
    y_threshold = 20  # pixels, adjust based on image scale
    matched_bicycles_with_distances = []

    for left_bicycle in bicycles_left:
        left_x, left_y = left_bicycle
        closest_bicycle = None
        min_dist = float('inf')

        for right_bicycle in bicycles_right:
            right_x, right_y = right_bicycle
            # Check if the y-coordinates are similar
            if abs(left_y - right_y) < y_threshold:
                # Calculate the distance (disparity)
                dist = abs(left_x - right_x)
                if dist < min_dist:
                    min_dist = dist
                    closest_bicycle = right_bicycle

        # If a match was found, calculate depth and add to list
        if closest_bicycle:
            right_x, _ = closest_bicycle
            disparity = abs(left_x - right_x)
            depth = (focal_length * baseline) / disparity if disparity != 0 else float('inf')
            matched_bicycles_with_distances.append((left_bicycle, depth))

    return matched_bicycles_with_distances

Kalman Filter Variables

In [17]:
x = np.zeros(6)  # Initial state: [x, y, vx, vy, ax, ay]
P = np.eye(6) * 1000  # Initial state covariance matrix (uncertainty)

# Measurement noise (based on how noisy your car detection is)
R = np.eye(2) * 0.1  # Measurement noise covariance (assumed small noise)

# Process noise (with acceleration in the model)
Q = np.eye(6) * 0.1  # Process noise covariance

In [16]:
curr_wp = 5
collision_happened = False
quitLoop = False

while curr_wp<len(route)-1:
    world.tick()
    
    while vehicle.get_transform().location.distance(route[curr_wp][0].transform.location) < 5:
        curr_wp += 1

    #Setup the object detection
    rightframe1 = rightcamera1_data['image']
    rightframe2 = rightcamera2_data['image']
    frontframe1 = frontcamera1_data['image']
    frontframe2 = frontcamera2_data['image']

    # frame1_gray = cv2.cvtColor(frame1, cv2.COLOR_BGRA2GRAY)
    # frame2_gray = cv2.cvtColor(frame2, cv2.COLOR_BGRA2GRAY)


    print("rightframe1 dtype:", rightframe1.dtype)
    print("rightframe2 dtype:", rightframe2.dtype)
    print("frontframe1 dtype:", frontframe1.dtype)
    print("frontframe2 dtype:", frontframe2.dtype)

    # Convert RGB image from BGRA to BGR
    rightframe1 = cv2.cvtColor(rightframe1, cv2.COLOR_BGRA2BGR)
    rightframe2 = cv2.cvtColor(rightframe2, cv2.COLOR_BGRA2BGR)
    frontframe1 = cv2.cvtColor(frontframe1, cv2.COLOR_BGRA2BGR)
    frontframe2 = cv2.cvtColor(frontframe2, cv2.COLOR_BGRA2BGR)
    #computeDepthMapSGBM(frame1_gray, frame2_gray)
    # Run the object detection model on the RGB frame
    results_right1 = model(rightframe1)
    results_right2 = model(rightframe2)
    
    results_front1 = model(frontframe1)
    results_front2 = model(frontframe2)
    
    bicycles_right1 = []
    bicycles_right2 = []
    bicycles_front1 = []
    bicycles_front2 = []



    for result in results_right1:
        for box in result.boxes:
            # Extract box coordinates and other details
            x1, y1, x2, y2 = box.xyxy[0]
            center_x = int((x1 + x2) / 2)  # x-center of the bicycle
            center_y = int((y1 + y2) / 2)  # y-center of the bicycle
            bicycles_right1.append((center_x, center_y))
            conf = box.conf[0]            # Confidence score
            cls = box.cls[0]
            cv2.rectangle(rightframe1, (int(x1), int(y1)), (int(x2), int(y2)), (0, 255, 0), 2)
            label = f"{model.names[int(cls)]}: {conf:.2f}"
            cv2.putText(rightframe1, label, (int(x1), int(y1) - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

            #break
              # Bounding box coordinates
    
    for result in results_right2:
        for box2 in result.boxes:
            # Extract box coordinates and other details
            x1, y1, x2, y2 = box2.xyxy[0]
            center_x = int((x1 + x2) / 2)
            center_y = int((y1 + y2) / 2)
            bicycles_right2.append((center_x, center_y))
            conf = box2.conf[0]            # Confidence score
            cls = box2.cls[0]

            cv2.rectangle(rightframe2, (int(x1), int(y1)), (int(x2), int(y2)), (0, 255, 0), 2)
            label = f"{model.names[int(cls)]}: {conf:.2f}"
            cv2.putText(rightframe2, label, (int(x1), int(y1) - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
    
    for result in results_front1:
        for box2 in result.boxes:
            # Extract box coordinates and other details
            x1, y1, x2, y2 = box2.xyxy[0]
            center_x = int((x1 + x2) / 2)
            center_y = int((y1 + y2) / 2)
            bicycles_front1.append((center_x, center_y))
            conf = box2.conf[0]            # Confidence score
            cls = box2.cls[0]

            cv2.rectangle(frontframe1, (int(x1), int(y1)), (int(x2), int(y2)), (0, 255, 0), 2)
            label = f"{model.names[int(cls)]}: {conf:.2f}"
            cv2.putText(frontframe1, label, (int(x1), int(y1) - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
    
    for result in results_front2:
        for box2 in result.boxes:
            # Extract box coordinates and other details
            x1, y1, x2, y2 = box2.xyxy[0]
            center_x = int((x1 + x2) / 2)
            center_y = int((y1 + y2) / 2)
            bicycles_front2.append((center_x, center_y))
            conf = box2.conf[0]            # Confidence score
            cls = box2.cls[0]

            cv2.rectangle(frontframe2, (int(x1), int(y1)), (int(x2), int(y2)), (0, 255, 0), 2)
            label = f"{model.names[int(cls)]}: {conf:.2f}"
            cv2.putText(frontframe2, label, (int(x1), int(y1) - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

    matched_bicycles_with_distances_right = match_bicycles_between_left_right(bicycles_right1, bicycles_right2)
    matched_bicycles_with_distances_front = match_bicycles_between_left_right(bicycles_front1, bicycles_front2)
    # Display distance for each matched bicycle on the left frame
    
    distance_front = 0
    distance_right = 0


    for (left_bicycle, distance) in matched_bicycles_with_distances_right:
        left_x, left_y = left_bicycle
        distance_labelright = f"Distance (right camera): {distance:.2f}m"
        distance_right = distance
        cv2.putText(rightframe1, distance_labelright, (left_x, left_y-20), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)
        print(distance_labelright)


    for (left_bicycle, distance) in matched_bicycles_with_distances_front:
        left_x, left_y = left_bicycle
        distance_front = distance
        distance_label = f"Distance (front camera): {distance:.2f}m"
        cv2.putText(frontframe1, distance_label, (left_x, left_y-20), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)
        print(distance_label)

    if distance_front > 0 and distance_right > 0:
        min_distance = np.min([distance_front, distance_right])
        max_distance = np.max([distance_front, distance_right])

        avg_distance = (min_distance*0.7 + max_distance*0.3)
    elif distance_front == 0:
        avg_distance = distance_right
    elif distance_right == 0:
        avg_distance = distance_front
    else:
        avg_distance = 1000
    
    print(f"Average distance: {avg_distance}")

    #Estimate the throttle 
    predicted_angle = get_angle(vehicle, route[curr_wp][0])
    v = vehicle.get_velocity()
    speed = round(3.6*math.sqrt(v.x**2 + v.y**2 + v.z**2),0)
    estimated_throttle = maintain_speed(speed)


    #Setup the steering angle
    if predicted_angle < -300:
        predicted_angle = predicted_angle+360
    elif predicted_angle > 300:
        predicted_angle = predicted_angle - 360
    steering_angle = predicted_angle

    if predicted_angle < -MAX_STEER_DEGREES:
        steering_angle = -MAX_STEER_DEGREES
    elif predicted_angle>MAX_STEER_DEGREES:
        steering_angle = MAX_STEER_DEGREES
    
    steering_angle = steering_angle/75


    #Apply the vehicle control to each of the two vehicles
    bicycle.apply_control(carla.VehicleControl(throttle=1))
    vehicle.apply_control(carla.VehicleControl(throttle = estimated_throttle, steer = steering_angle))
    
    # speed_text = f"{speed} km/h"
    # cv2.putText(frame1, speed_text, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 255), 2)

    cv2.imshow('Front1 camera',frontframe1)
    #cv2.imshow("camera2", rightframe1)
    # Exit loop on 'q' key press
    if cv2.waitKey(1) == ord('q') or collision_happened:
        break

    print('------------------')
    print('These are the parameters that we might need')
    try:
        print(f"left: {distance_labelright}")
    except:
        print("Még nincs jobbos bicikli")
    try:
        print(f"normal: {distance_label}")
    except:
        None

    #Check if a collision happenned
    if collision_happened:
        break
    
cv2.destroyAllWindows() 
vehicle.apply_control(carla.VehicleControl(throttle=0, steer=0, brake=1))
bicycle.apply_control(carla.VehicleControl(throttle=0, steer=0, brake=1))
destroy()
bicycleorigin()
carorigin()

rightframe1 dtype: uint8
rightframe2 dtype: uint8
frontframe1 dtype: uint8
frontframe2 dtype: uint8



0: 384x640 (no detections), 78.7ms
Speed: 1.5ms preprocess, 78.7ms inference, 1.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 65.0ms
Speed: 2.0ms preprocess, 65.0ms inference, 0.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 Bicycle, 67.5ms
Speed: 2.0ms preprocess, 67.5ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 Bicycle, 63.2ms
Speed: 2.0ms preprocess, 63.2ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 62.0ms
Speed: 2.0ms preprocess, 62.0ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 62.1ms
Speed: 2.0ms preprocess, 62.1ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)



Distance (front camera): 9.41m
Average distance: 9.411764705882355
------------------
These are the parameters that we might need
Még nincs jobbos bicikli
normal: Distance (front camera): 9.41m
rightframe1 dtype: uint8
rightframe2 dtype: uint8
frontframe1 dtype: uint8
frontframe2 dtype: uint8


0: 384x640 1 Bicycle, 66.1ms
Speed: 1.5ms preprocess, 66.1ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 Bicycle, 64.6ms
Speed: 1.0ms preprocess, 64.6ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 63.1ms
Speed: 1.0ms preprocess, 63.1ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)



Distance (front camera): 9.41m
Average distance: 9.411764705882355
------------------
These are the parameters that we might need
Még nincs jobbos bicikli
normal: Distance (front camera): 9.41m
rightframe1 dtype: uint8
rightframe2 dtype: uint8
frontframe1 dtype: uint8
frontframe2 dtype: uint8


...

Average distance: 0
------------------
These are the parameters that we might need
Még nincs jobbos bicikli
normal: Distance (front camera): 4.32m
rightframe1 dtype: uint8
rightframe2 dtype: uint8
frontframe1 dtype: uint8
frontframe2 dtype: uint8


0: 384x640 (no detections), 60.1ms
Speed: 1.0ms preprocess, 60.1ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 62.6ms
Speed: 1.0ms preprocess, 62.6ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 60.1ms
Speed: 2.0ms preprocess, 60.1ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)



Average distance: 0
------------------
These are the parameters that we might need
Még nincs jobbos bicikli
normal: Distance (front camera): 4.32m
rightframe1 dtype: uint8
rightframe2 dtype: uint8
frontframe1 dtype: uint8
frontframe2 dtype: uint8


0: 384x640 (no detections), 65.2ms
Speed: 2.0ms preprocess, 65.2ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 63.5ms
Speed: 1.0ms preprocess, 63.5ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 61.9ms
Speed: 1.0ms preprocess, 61.9ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)


Average distance: 0
------------------
These are the parameters that we might need
Még nincs jobbos bicikli
normal: Distance (front camera): 4.32m
